# NOTE: run the below cell when executing the notebook for the first time

In [3]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
import glob
import math
import re
import sys
from collections import defaultdict
from functools import reduce
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
STOPWORDS = set(stopwords.words("english"))
CORPUS = "corpus/*"


In [3]:
STOPWORDS

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

# some variable declarations that we will be using later on

In [4]:
# Each document has an id, and these are the keys in the following dict.
# The values are the corresponding filenames.
document_filenames = dict()

# The size of the corpus
N = 0

# vocabulary: a set to contain all unique terms (i.e. words) in the corpus
vocabulary = set()

# postings: a defaultdict whose keys are terms, and whose corresponding values are
# "postings list" for that term, i.e., the list of documents the term appears in.
#
# postings list: dict whose keys are the document ids of documents that the term appears
# in, with corresponding values equal to the frequency with which the term occurs
# in the document.
#
# As a result, postings[term] is the postings list for term, and
# postings[term][id] is the frequency with which term appears in document id.
postings = defaultdict(dict)

# document_frequency: a defaultdict whose keys are terms, with corresponding
# values equal to the number of documents which contain the key, i.e.
# the document frequency.
document_frequency = defaultdict(int)

# length: a defaultdict whose keys are document ids, with values equal
# to the Euclidean length of the corresponding document vector.
length = defaultdict(float)

In [5]:
def get_corpus():
    global document_filenames, N

    # Fetch list of document names in corpus
    documents = glob.glob(CORPUS)

    # Set size of corpus
    N = len(documents)

    # Dictionary having doc id as key and document name as value
    document_filenames = dict(zip(range(N), documents))
    print(document_filenames)


# here we have 3 documents in our corpus

In [6]:
get_corpus()

{0: 'corpus/the_hobbit.txt', 1: 'corpus/silmarillion.txt', 2: 'corpus/lotr.txt', 3: 'corpus/rainbows_end.txt'}


# helper functions

In [9]:

def remove_special_characters(text):
    """ Removes special characters using regex substitution """
    regex = re.compile(r"[^a-zA-Z0-9\s]")
    return re.sub(regex, "", text)

In [10]:
def remove_digits(text):
    """ Removes digits using regex substitution """
    regex = re.compile(r"\d")
    return re.sub(regex, "", text)

In [11]:
def intersection(sets):
    """Returns the intersection of all sets in the list sets. Requires
    that the list sets contains at least one element, otherwise it
    raises an error

    :param sets: list of sets whose intersection we want to find
    """
    return reduce(set.intersection, [s for s in sets])

In [12]:
def tokenize(document):
    """Returns a list whose elements are the separate terms in document

    :param document: document to tokenize
    :returns: list of lowercased tokens after removing stopwords
    """
    # Tokenize text into terms
    terms = word_tokenize(document)

    # Remove stopwords and convert remaining terms to lowercase
    terms = [term.lower() for term in terms if term not in STOPWORDS]

    return terms

# function to create postings

In [13]:
def initialize_terms_and_postings():
    """Reads in each document in document_filenames, splits it into a
    list of terms (i.e., tokenizes it), adds new terms to the global
    vocabulary, and adds the document to the posting list for each
    term, with value equal to the frequency of the term in the
    document
    """
    global vocabulary, postings
    for id in document_filenames:

        # Read the document
        with open(document_filenames[id], "r") as f:
            document = f.read()

        # Remove all special characters from the document
        document = remove_special_characters(document)

        # Remove digits from the document
        document = remove_digits(document)

        # Tokenize the document
        terms = tokenize(document)

        # Remove duplicates from the terms
        unique_terms = set(terms)

        # Add unique terms to the vocabulary
        vocabulary = vocabulary.union(unique_terms)

        # For every unique term
        for term in unique_terms:

            # The value is the frequency of the term in the document
            postings[term][id] = terms.count(term)
            


In [16]:
initialize_terms_and_postings()

# we first created the posting, which tells us, in which document the term is present and in how much frequency it is present

### Ex. we can see below that the word `well` is present in document 0 : 3 times, and 

In [17]:
postings

defaultdict(dict,
            {'well': {0: 3, 1: 1},
             'stops': {0: 1},
             'perilous': {0: 1, 2: 1},
             'unexpected': {0: 1, 3: 1},
             'fortuneseeking': {0: 1},
             'nasty': {0: 1},
             'it': {0: 1},
             'lord': {0: 1, 2: 3},
             'trilogy': {0: 1},
             'comeand': {0: 1},
             'follows': {0: 1},
             'giant': {0: 1},
             'jrr': {0: 1},
             'certainly': {0: 1},
             'elsewhere': {0: 1},
             'look': {0: 1},
             'lonely': {0: 1},
             'tolkiens': {0: 1},
             'reclaim': {0: 1},
             'primed': {0: 1},
             'doorstep': {0: 1},
             'meet': {0: 1},
             'by': {0: 1},
             'set': {0: 1},
             'returns': {0: 1},
             'dragon': {0: 1},
             'iron': {0: 1},
             'question': {0: 1},
             'altogether': {0: 1},
             'return': {0: 1},
             'childr

In [20]:
def initialize_document_frequencies():
    """For each term in the vocabulary, count the number of documents
    it appears in, and store the value in document_frequncy[term]
    """
    global document_frequency
    for term in vocabulary:
        document_frequency[term] = len(postings[term])

In [21]:
initialize_document_frequencies()

# Here we can see that  the term mountains appeared in only 1 document overall, where as the word filled appeared 2 times

In [22]:
document_frequency

defaultdict(int,
            {'vain': 1,
             'law': 1,
             'profundity': 1,
             'hes': 1,
             'mighty': 1,
             'tall': 1,
             'there': 1,
             'fell': 1,
             'chip': 1,
             'contended': 1,
             'giant': 1,
             'spoke': 1,
             'disturbed': 1,
             'organs': 1,
             'accessthrough': 1,
             'turbulent': 1,
             'look': 1,
             'being': 1,
             'now': 1,
             'reclaim': 1,
             'primed': 1,
             'doorstep': 1,
             'drown': 1,
             'others': 2,
             'void': 1,
             'alzheimers': 1,
             'decline': 1,
             'dragon': 1,
             'tremor': 1,
             'altogether': 1,
             'propounding': 1,
             'seat': 1,
             'elvensmiths': 1,
             'gifts': 2,
             'children': 2,
             'fervently': 1,
             'contextthrough'

# below is the vocabulary, which is collection of all the unique terms from all of the 3 documents

In [23]:
vocabulary

{'abyss',
 'accessthrough',
 'accord',
 'achieved',
 'across',
 'adorning',
 'adults',
 'adventure',
 'adventures',
 'after',
 'age',
 'ages',
 'ainur',
 'alone',
 'along',
 'altogether',
 'alzheimers',
 'amazed',
 'amid',
 'among',
 'analysts',
 'ancestral',
 'ancient',
 'and',
 'another',
 'arda',
 'aright',
 'arose',
 'around',
 'arrived',
 'as',
 'assigned',
 'assuaged',
 'attempts',
 'attune',
 'aught',
 'available',
 'baffles',
 'baggins',
 'bare',
 'bearded',
 'beautiful',
 'beauty',
 'becomes',
 'beer',
 'began',
 'beginning',
 'begins',
 'begun',
 'behold',
 'being',
 'belongs',
 'bequeathing',
 'best',
 'beyond',
 'bigger',
 'bilbo',
 'bind',
 'birthday',
 'blended',
 'books',
 'boromir',
 'bowed',
 'braying',
 'brethren',
 'bring',
 'built',
 'builtin',
 'burglar',
 'but',
 'by',
 'called',
 'came',
 'casting',
 'cease',
 'ceased',
 'certainly',
 'chance',
 'change',
 'changed',
 'chiefly',
 'children',
 'chip',
 'choice',
 'choirs',
 'chord',
 'clamorous',
 'climax',
 'clot

In [27]:
def term_frequency(term, id):
    """Returns the term frequency of term in document id.  If the term
    isn't in the document, then return 0

    :param term: term whose tf we want to find
    :param id: document to find in
    :returns: term frequency
    """
    if id in postings[term]:
        return postings[term][id]
    else:
        return 0.0

In [28]:

def initialize_lengths():
    """ Computes the length for each document """
    global length
    for id in document_filenames:
        l = 0
        for term in vocabulary:
            l += term_frequency(term, id) ** 2
        length[id] = math.sqrt(l)

In [29]:
initialize_lengths()

In [30]:
length

defaultdict(float,
            {0: 16.852299546352718,
             1: 43.474130238568314,
             2: 16.401219466856727,
             3: 17.11724276862369})

In [31]:
def inverse_document_frequency(term):
    """Returns the inverse document frequency of term.  Note that if
    term isn't in the vocabulary then it returns 0, by convention

    :param term: term whose idf we want to find
    :returns: inverse document frequency
    """
    if term in vocabulary:
        return math.log(N / document_frequency[term], 2)
    else:
        return 0.0


In [32]:

def print_scores(scores):
    """Prints scores in a tabular format with two columns like
    | Score | Document |
    --------------------
    | 0.523 | foo      |
    --------------------

    :param scores: list of (id, score)
    """
    print("-" * 42)
    print("| %s | %-30s |" % ("Score", "Document"))
    print("-" * 42)

    for (id, score) in scores:
        if score != 0.0:
            print("| %s | %-30s |" % (str(score)[:5], document_filenames[id]))

    print("-" * 42, end="\n\n")


In [35]:

def do_search():
    """Asks the user what they would like to search for, and returns a
    list of relevant documents, in decreasing order of cosine similarity
    """
    query = tokenize(input("Search query >> "))

    # Exit if query is empty
    if query == []:
        sys.exit()

    scores = sorted(
        [(id, similarity(query, id)) for id in range(N)], key=lambda x: x[1],reverse=True,)

    return scores


In [36]:

def similarity(query, id):
    """Returns the cosine similarity between query and document id.
    Note that we don't bother dividing by the length of the query
    vector, since this doesn't make any difference to the ordering of
    search results

    :param query: list of tokens in query
    :param id: document ID
    :returns: similarity of document with query
    """
    similarity = 0.0

    for term in query:

        if term in vocabulary:

            # For every term in query which is also in vocabulary,
            # calculate tf-idf score of the term and add to similarity
            similarity += term_frequency(term, id) * inverse_document_frequency(term)

    similarity = similarity / length[id]

    return similarity



In [37]:
def main():
    # Get details about corpus
    get_corpus()

    # Initialise terms and postings for the corpus
    initialize_terms_and_postings()

    # Set document frequencies for all terms
    initialize_document_frequencies()

    # Set document vector lengths
    initialize_lengths()

    # Allow for search
    while True:

        # Retrieve sorted list of ranked documents
        scores = do_search()

        # Print the results in tabular format
        print_scores(scores)

# now you can do the query search and output will be show the relevant documents with decreasing cosine scores

In [ ]:
main()

{0: 'corpus/the_hobbit.txt', 1: 'corpus/silmarillion.txt', 2: 'corpus/lotr.txt', 3: 'corpus/rainbows_end.txt'}
Search query >> lord of the rings
------------------------------------------
| Score | Document                       |
------------------------------------------
| 0.365 | corpus/lotr.txt                |
| 0.118 | corpus/the_hobbit.txt          |
------------------------------------------

Search query >> ring was found in san diego
------------------------------------------
| Score | Document                       |
------------------------------------------
| 0.548 | corpus/lotr.txt                |
| 0.233 | corpus/rainbows_end.txt        |
| 0.059 | corpus/the_hobbit.txt          |
| 0.046 | corpus/silmarillion.txt        |
------------------------------------------

